In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
df.drop("RID",axis=1, inplace = True) #RID는 그냥 Index라서 삭제

In [2]:
df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


entropy를 구하는 식은 $-\sum_{k = 1}^{m}p_klog_2{p_k}$이다.

우선 Series.value_counts함수를 사용하여 주어진 feature의 class별 확률을 구한다.

이후 초기값 entropy = 0이후 계산식에 따라 for문을 돌며 $plog_2{p}$를 빼주어 계산한다.

In [3]:
def getEntropy(df, feature):
    entropy = 0 # entropy값 초기화
    ps = df[feature].value_counts() / len(df)
    # pandas.Series.value_counts함수를 이용하여 feature의 class별 확률을 계산
    for p in ps:
        entropy -= p * np.log2(p)
        # entropy에 p * log2(p)를 빼주어 최종 entropy계산
    return entropy

In [4]:
getEntropy(df, "class_buys_computer")

0.9402859586706311

의사결정나무는 분기를 위한 feature를 결정하기 위해서 entropy을 기준으로 활용한다.

그 기준 Information Gain이란 상위노드의 entropy에서 하위노드의 entropy를 뺀 값이다.

Information_Gain(S, F) $= e(S) - \sum_{f\in{F}} \frac{S_f}{S}e(S_f)$이다.

식을 풀어보면 분기 전 노드의 entropy에서 분기할 feature의 class별 entropy를 가중평균한 값을 빼준다. 

즉, Information Gain은 분기 전 entropy와 분기 후 entropy의 차이로 그 차이가 클수록 분류가 잘 되고있음을 뜻한다.

In [5]:
def getGainA(df, feature) :
    
    def entropy_by_target(df, feature, feature_under):
        entropy = 0 #entropy값 초기화
        n = len(df)
        # 추후 feature_under의 class별 가중평균을 위해 전체 샘플 수 저장
        unique = df[feature_under].unique()
        # feature_under의 class를 unique변수에 저장
        
        for i in unique: # feature_under의 class별 for
            tmp_df = df[df[feature_under].isin([i])] 
            # isin함수를 사용하여 feature_under의 class가 i인 것만 저장
            entropy += len(tmp_df) / n * getEntropy(tmp_df, feature)
            # 위에서 정의한 getEntropy함수를 통해 target feature의 entropy를 가중평균함
        return entropy
        
    info_D = getEntropy(df, feature) # 목표변수 Feature에 대한 Info(Entropy)를 구함
    columns = list(df.loc[:, df.columns != feature])
    # 목표변수를 제외한 나머지 설명변수들을 리스트 형태로 저장
    result = dict() # 결과 dictionary
    
    for feature_under in columns:
        result[feature_under] = info_D - entropy_by_target(df, feature, feature_under)
        # entropy_by_target함수를 이용하여 entropy를 계산한 후 info_D에서 빼서 information gain을 계산
    return result

In [6]:
getGainA(df, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}